# 🦌 Make the ELK App Interactive 🕹️
This notebook shows how you can make the ELK App work dynamically with various types of graphs

In [ ]:
import random

import ipywidgets as W
import networkx as nx

from pprint import pprint
from collections import defaultdict

import elk
from elk import nx as elknx

In [ ]:
elk_app = elk.Elk(
    style = {" ": {'height': "100%"}},
    transformer=elknx.transformer.XELK(),
)
elk_app

In [ ]:
@W.interact
def make_graph(
    number_of_nodes=(5, 100),
    percent_of_edges=(1, 99),
    hierarchical=False,
    seed=(0, 1024),
    debug=False,
    fit=False,
    padding=(0, 100),
    port_scale=(1, 20),
    text_scale=(1, 20),
    label_offset=(1, 10)
):
    tree = None
    if hierarchical:
        tree = nx.generators.trees.random_tree(number_of_nodes, seed=seed)
        
    number_of_edges = max(1, int(number_of_nodes * 0.01 * percent_of_edges))

    graph = nx.generators.random_graphs.barabasi_albert_graph(
        n=number_of_nodes,
        m=number_of_edges,
        seed=seed,
    )
    
    for edge in sorted(graph.edges):
        graph.edges[edge]["sourcePort"] = edge
        graph.edges[edge]["targetPort"] = edge
    
    elk_app.transformer.port_scale = port_scale
    elk_app.transformer.label_offset = label_offset
    elk_app.transformer.text_scale = text_scale
    elk_app.transformer.source = (
        graph,
        tree,
    )
    if fit:
        elk_app.diagram.fit('root', padding=padding)
    
    if debug:
        counter = defaultdict(list)
        [counter[src].append(tgt) for src, tgt in graph.edges]
        pprint(dict(counter))
        pprint(elk_app.transformer.value["edges"])

## 🦌 Learn More 📖 

- [🦌 Introducing ELK 👋](./00_Introduction.ipynb)
- [🦌 Linking ELK Diagrams 🔗](./01_Linking.ipynb)
- [🦌 ELK Transformer 🤖](./02_Transformer.ipynb)
- [🦌 ELK App 🚀](./03_App.ipynb)
- [🦌 Interactive ELK App 🕹️](./04_Interactive.ipynb)